In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#!nvidia-smi

Wed Apr 14 15:58:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    62W / 149W |    124MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import sys
sys.path.append('/content/gdrive/MyDrive/codigo_modelo') # tiene bien la barra

In [3]:
from GPU_rot_hamming import rot_hamming
from corr import corr
from gridness_2019 import gridness_2019
from GPU_autoc_fast_2D import autoc_fast_2D

In [4]:
from plot_all_grid import plot_all_grid

In [68]:
import tensorflow as tf 
tf.test.gpu_device_name() # para chequear que sea gpu

'/device:GPU:0'

In [5]:
DATA_HOME=r"/content/gdrive/MyDrive/codigo_modelo/"
from pathlib import Path
DATA_HOME = Path(DATA_HOME)

In [6]:
import numpy as np
import time
from scipy import stats

In [7]:
NmECs=10
NIs=15
NmEC = NmECs**2 # cantidad de grid cells. Hay 100 grid cells
vel = 0.6 # velocidad a la que corre la rata (en cm/s)
placeSTD = 7*(vel**0.5) # cm, es el ancho de las place cells
s0=0.6
a0=0.1
porcent=int((1-s0)*NmEC)-1
runningAverageWeight = 0.5
un_run=(1-runningAverageWeight)
NI=NIs**2 # es el numero de inputs: tengo 225 place cells
maxWeight= 15/NI
A=2
sigmaW=2
epsilon = 0.00023/vel**(1.5) #constante de aprendizaje
phiSat = 30  # Saturation Phi
b2=vel/placeSTD*0.4 #Speed of fall of activity

In [8]:
cPx=tf.range(0,100,100/NIs)+50/NIs # 100cm es el lado del ambiente cuadrado
cellPositionsX=tf.repeat(cPx, [NIs])
cellPositionsY=tf.tile(cPx, [NIs])
# circular W for ring attractor SOLO PARA NUMERO PAR DE GRID CELLS
W0=tf.math.exp(-(tf.concat(( tf.range(0,NmEC/2,1),tf.range(NmEC/2,0,-1)), 0))**2/2/sigmaW**2)
W=tf.Variable(tf.zeros([NmEC,NmEC]))
for jW in range(NmEC):
    W=W[jW,:].assign(tf.concat(( W0[-1-jW+1:],W0[:-1-jW+1]), 0))
#W normalization
W=W.assign(W-tf.eye(NmEC))
W=W.assign(A*W/(sum(W[1,:]**2)**0.5))
Wautoc=tf.Variable(tf.zeros([2*NmECs-1,2*NmECs-1]))
for i in range(NmEC):
    Wautoc=Wautoc.assign(Wautoc+autoc_fast_2D(tf.transpose(tf.reshape(W[i,:],(NmECs, NmECs)))))    
Wautoc=Wautoc/NmEC

In [16]:
#parametros simulacion
ms=40 #bins per map. este valor da bins de 2.5 cm, es 100 cm/40 bins = 2.5 cm por bin 
binsize=100/ms
smap=0.05 #running average for map
un_smap=(1-smap)
simdur = 12000000 #tiempo total de la simulacion
pgr = 50000 #12000000-2
cociente=int(simdur/pgr)
# variables para gridness
angbins=tf.range(0,np.pi-0.0001,np.pi/120)
abins=tf.range(-ms+1,ms)
tbins=tf.range(0,len(abins)+0.5,0.5)
rx=np.array(tf.tensordot(tf.cos(angbins),tbins, axes=0))
ry=np.array(tf.tensordot(tf.sin(angbins),tbins, axes=0))
rx[rx>max(abins)]=np.nan
rx[rx<min(abins)]=np.nan
ry[ry>max(abins)]=np.nan
ry[ry<min(abins)]=np.nan
rx=tf.Variable(rx)
ry=tf.Variable(ry)
b=tf.math.exp(1j*np.array(angbins)*6) #algo con cualquier fase

In [72]:
for rep in range(1):
    mapp=tf.Variable(tf.zeros([NmEC,ms*ms]))
    mymap=tf.transpose(tf.reshape(mapp[0,:],(ms, ms)))
   # rot_ha=rot_hamming(autoc_fast_2D(mymap))
    mngr=np.empty(cociente) # mean gridness vs time  
    mnsp=np.empty(cociente) # mean spacing vs time   
    gridness_mean=np.empty(cociente) # population gridness vs time 
    spacing_mean=np.empty(cociente) # population spacing vs time   
    mnal=np.empty(cociente) #alignment vs time 
    mnWJ=np.empty(cociente) #functional connectivity similar to actual vs time  
    J=np.random.rand(NmEC,NI).astype(np.float32)
    J=maxWeight*tf.linalg.normalize(J,axis=1)[0]
    #simulacion
    start_time = time.time()
    ffacc=np.empty(cociente) 
    rracc=np.empty(cociente)  
    meanJW=0
    mxphi=0
    phi=tf.zeros([NmEC,1]) #output
    rec=phi #np.zeros(NmEC) #recurrentes
    r_act=phi #np.zeros(NmEC) #activacion
    r_inact=tf.ones([NmEC,1]) #inactivacion
    r=tf.zeros([NI,1]) #vector input
    h=np.random.rand(NmEC,1) #input de place a grid
    meanphi=phi
    meanr=r
    x=np.zeros(simdur) # reemplazar con empty # Posicion X [cm]
    x[0]=50
    y=np.zeros(simdur) # reemplazar con empty
    y[0]=50
    mydir=np.pi/4
    contador=0
    for t in range(1,simdur): #Variable temporal
        mydir=(mydir+0.3*np.random.randn()) 
        x[t]=(x[t-1]+vel*np.cos(mydir))
        y[t]=(y[t-1]+vel*np.sin(mydir))
        while (x[t]<0) or (x[t]>100) or (y[t]<0) or (y[t]>100):
            mydir=(mydir+0.1*np.random.randn())
            x[t]=(x[t-1]+vel*np.cos(mydir))
            y[t]=(y[t-1]+vel*np.sin(mydir))
        #activacion de place cell
        r=phiSat*tf.math.exp(-((x[t]-cellPositionsX)**2+(y[t]-cellPositionsY)**2)/(2*placeSTD**2))
        r_act=(h-r_inact)
        r_inact=(r_inact+b2*r_act)
        #inputs de place cells
        hff=tf.matmul(J,tf.reshape(r,(len(r),1))) #J@r es mas lento
        #agregar recurrente
        h=(hff+rec)
        #aux=np.sort(r_act) #sorted(list(r_act)) es mas lento
        theta=np.sort(r_act)[porcent] # es int no round, ya lo chequee
        thresholdedActivity =(np.array(r_act)-theta)*((np.array(r_act)-theta)>0)
        phi = thresholdedActivity*a0/np.mean(thresholdedActivity)
        meanphi = (meanphi*un_run + runningAverageWeight*phi)
        meanr = (meanr*un_run + runningAverageWeight*r)
        myx=round((x[t]+binsize/2)/binsize)
        myy=round((y[t]+binsize/2)/binsize)
        subind=ms*(myy-1)+myx-1
        mapp=mapp[:,subind].assign(smap*phi+un_smap*mapp[:,subind])
        #learning
        J = (J + epsilon*(np.outer(phi,r) - np.outer(meanphi,meanr)))
        #J = (J + epsilon*(tf.tensordot(meanphi,meanr, axes=0) - tf.tensordot(meanphi,meanr, axes=0)))
       # J[J<0] = 0
        #normalizacion euclidea
        J=maxWeight*tf.linalg.normalize(J,axis=1)[0]
     #   rec=np.dot(W,phi)
        # PLOTS
        if np.mod(t,pgr)==0:
            print('Tardo {} segundos de cálculo'.format((time.time() - start_time)))
            start_time = time.time()
            meanJW=(0.999*meanJW+0.001*J.mean()/W.mean())
            mxphi=(0.999*mxphi+0.001*max(phi))
            ff=np.mean(hff)
            rr=np.mean(rec)
            meanffrr=np.mean(hff-rec)
            map_sparseness=np.mean((mapp.mean(1)**2)/(mapp**2).mean(1))
            print('J/W: ' + str(meanJW) + ',  r: ' + str(np.mean(meanphi)) + ', rmx: ' + str(mxphi) + ',  hFF-hRR: ' + str(meanffrr) + ', Spar: ' + str(map_sparseness) +  ', FF80%: ' + str(ff) + ', RR80%: ' + str(rr) )
            gridness = np.empty(NmEC)
            spacing= np.empty(NmEC) 
            X=np.empty((6,NmEC))
            Y=np.empty((6,NmEC))
            V=np.empty((6,NmEC))
            autocr=np.empty(((ms+ms-1)**2,NmEC))
            mcon=corr(J.transpose(),J.transpose())
            mcon=(mcon-np.identity(mcon.shape[0]))
            Jautoc=np.zeros(2*NmECs-1)
            for i in range(NmEC):
                Jautoc=(Jautoc+autoc_fast_2D(np.reshape(mcon[i,:], (NmECs, NmECs), order="F")).real)   
            Jautoc=Jautoc/NmEC
            for i in range(NmEC):
                mymap=np.reshape(mapp[i,:], (ms, ms), order="F") #.astype(np.float32)
                gridness[i], spacing[i],  X[:,i], Y[:,i], V[:,i], autocr[:,i] = gridness_2019(autoc_fast_2D(mymap),abins,rot_ha,rx,ry,angbins,b,tbins)
            mean_autoc=np.nanmean(autocr,axis=1) # promedio por filas
            gridness_mean[contador], spacing_mean[contador] = gridness_2019(np.reshape(mean_autoc, (2*ms-1, 2*ms-1), order="F"), abins,rot_ha,rx,ry,angbins,b,tbins)[:2] #gridness of the mean autoc
            mngr[contador]=np.nanmean(gridness) 
            mnsp[contador]=np.mean(spacing)
            ffacc[contador]=ff
            rracc[contador]=rr
            aux1=np.delete(mcon, np.arange(0,NmEC**2,NmEC+1), None)
            aux2=np.delete(W, np.arange(0,NmEC**2,NmEC+1), None)
            #mnWJ[contador]=corr(np.reshape(aux1, (len(aux1), 1)), np.reshape(aux2, (len(aux2), 1))) 
            mnWJ[contador]=stats.pearsonr(aux1,aux2)[0]
            hs=np.histogram(np.arctan2(Y.ravel(),X.ravel()), bins=np.arange(-np.pi,16*np.pi/15,np.pi/15))[0]
            hs=np.reshape(hs, (5, 6), order="F")
            hs=np.concatenate((hs[:,0:3], hs[:,1:4], hs[:,2:5], hs[:,3:6]), axis=0)
            mnal[contador]=(max(hs.sum(axis=1))/len(Y.ravel())-1/10)*2
            plot_all_grid(mapp,gridness,X,Y,V,mngr,mnWJ,W,mnal,gridness_mean,pgr,Wautoc,Jautoc,ffacc,rracc,ms,NmEC,ff,rr,contador)
            contador+=1
            print('Tardo {} segundos de cálculo'.format((time.time() - start_time)))
            start_time = time.time()
    np.savez(str(DATA_HOME / '{}'.format(rep)), mngr=mngr, gridness_mean=gridness_mean, mnsp=mnsp, spacing_mean=spacing_mean, ffacc=ffacc, rracc=rracc, J=J, W=W)

UnimplementedError: ignored

In [66]:
#W.shape
phi.shape

(100,)